# <center>ANALYSIS OF RUSSIAN PROPAGANDA ON TWITTER<center>

# Daily summarization

For daily summarization of Russian propaganda I used the Hugging Face transformers library to create a summarization pipeline. The transformers library is a library for Natural Language Processing (NLP) tasks developed by Hugging Face, which uses pre-trained transformer models like GPT-3 to perform various NLP tasks efficiently.

In [8]:
date_r  = r_trolls[r_trolls['date'] == '2/24/2022']

text_r = ''
for tweet in date_r['tweet']:
    text_r += tweet

## Pre processing

In [9]:
text_r = re.sub(r"http\S+", "", text_r)
text_r = text_r.replace("@", "")
text_r = text_r.replace("#", "")
text_r = text_r.replace(".", ".<eos>")
text_r = text_r.replace("!", ".<eos>")
text_r = text_r.replace("?", ".<eos>")

sentences_r = text_r.split("<eos>")

new_sentences_r = []
for sen in sentences_r:
    if len(sen) < 5:
        continue
    else:
        new_sentences_r.append(re.sub(' +', ' ', sen)) # remove extra spaces

In [10]:
new_sentences_r

[' President Vladimir Putin: We have to take bold and immediate action.',
 ' The People\x92s Republics of Donbass have asked Russia for help.',
 ' In accordance with Article 51 of the UN Charter I made a decision to carry out a special military operation.',
 ' Vladimir Putin: Over the past 30 years we have been patiently trying to come to an agreement with NATO regarding the principles of equal and indivisible security in Europe.',
 ' In response, we faced either cynical deception and lies or attempts at pressure and blackmail.',
 ' Putin: For 8 years Russia has been doing everything possible to settle the situation by peaceful means.',
 ' Everything was in vain.',
 ' It became impossible to tolerate.',
 ' We had to stop the atrocity &amp; genocide of millions of people in Donbass who pinned their hopes on Russia.',
 ' Vladimir Putin: A veritable \x93empire of lies\x94 has been created inside the US in recent years.',
 ' All its satellites not only obediently say yes to and parrot it a

### Dividing the input text into smaller chunks

After this code is executed, chunks_r list will contain a list of text chunks, and each chunk will have a length less than or equal to 500. These chunks can then be fed into the summarization pipeline (summarizer2) to obtain summaries for each chunk, effectively summarizing the original text in smaller parts.

In [11]:
from transformers import pipeline

summarizer2 = pipeline("summarization")

max_chunk = 500
current_chunk = 0 
chunks_r = []
for sentence in new_sentences_r:
    if len(chunks_r) == current_chunk + 1: 
        if len(chunks_r[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks_r[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks_r.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks_r.append(sentence.split(' '))

for chunk_id in range(len(chunks_r)):
    chunks_r[chunk_id] = ' '.join(chunks_r[chunk_id])

C:\Users\Asus\anaconda3\envs\notebook_myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


0


### Summarization

In [12]:
result_r = summarizer2(chunks_r, max_length=120, min_length=30, do_sample=False)

Your max_length is set to 120, but you input_length is only 44. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)


In [13]:
result_r

[{'summary_text': " Putin: 'We have to take bold and immediate action. The People\x92s Republics of Donbass have asked Russia for help. In accordance with Article 51 of the UN Charter I made a decision to carry out a special military operation' Putin: For the US &amp; its allies, it is a policy of containing Russia, with obvious geopolitical dividends ."},
 {'summary_text': " Russian flag was raised in Kakhovka, in the Kherson region . President Vladimir Putin and Prime Minister ImranKhanPTI held a meeting in Moscow . LPR People's Militia stated that the soldiers of the 57th Separate Motorized Infantry Brigade of the Armed Forces of Ukraine voluntarily laid down their arms ."},
 {'summary_text': " Putin: Focused on their own goals, leading NATO countries are supporting far-right nationalists and neo-Nazis in Ukraine . Lybia didn't threat a single NATO member, nor Iraq, nor Yugoslavia.  Article 5 is for members.  Not even Afganistan posed a threat.  NATO didn't admit Ukraine in for a re

In [14]:
propaganda_summary = ''
for i in range(len(result_r)):
    propaganda_summary += result_r[i]['summary_text']
    
propaganda_summary

" Putin: 'We have to take bold and immediate action. The People\x92s Republics of Donbass have asked Russia for help. In accordance with Article 51 of the UN Charter I made a decision to carry out a special military operation' Putin: For the US &amp; its allies, it is a policy of containing Russia, with obvious geopolitical dividends . Russian flag was raised in Kakhovka, in the Kherson region . President Vladimir Putin and Prime Minister ImranKhanPTI held a meeting in Moscow . LPR People's Militia stated that the soldiers of the 57th Separate Motorized Infantry Brigade of the Armed Forces of Ukraine voluntarily laid down their arms . Putin: Focused on their own goals, leading NATO countries are supporting far-right nationalists and neo-Nazis in Ukraine . Lybia didn't threat a single NATO member, nor Iraq, nor Yugoslavia.  Article 5 is for members.  Not even Afganistan posed a threat.  NATO didn't admit Ukraine in for a reason.  Keeping Germany under control was worthwhile killing Ukra